In [524]:
# Importing necessary libraries

import numpy as np
import pandas as pd
import os
import cmath


In [525]:
# Reading bus data into panda dataframes

def r_buses(file):
    df_buses = pd.read_csv(file, sep=";")
    V = df_buses.V.to_numpy()
    delta = df_buses.Angle.to_numpy()
    P = df_buses.P_gen.to_numpy()
    Q = df_buses.Q_gen.to_numpy()
    bus_type = df_buses.BusType.to_numpy()
    return V, delta, P, Q, bus_type


V, delta, P, Q, bus_type = r_buses('Busdata.csv')

Q_max =[0.5, "", "", "", ""]

print(V)
print(delta)
print(P)
print(Q)
print(bus_type)


# Creating additional vectors that will be used and updaed during the calculations
V_calc, delta_calc, P_calc, Q_calc, bus_type = r_buses('Busdata.csv')

for x in range(len(V_calc)):
    if (np.isnan(V_calc[x])):
        V_calc[x] = 1
    if(np.isnan(delta_calc[x])):
        delta_calc[x] = 0



[ 1. nan  1. nan nan]
[nan nan  0. nan nan]
[ 1.  -0.6  nan -0.6 -0.5]
[ nan -0.3  nan -0.2 -0.4]
[1 2 0 2 2]


In [526]:
# Function to create the Y-bus matrix
# shape needs to be specified per now. Fix this?
def Ybus(file, shape):
    df_impedances = pd.read_csv(file, sep=";")
    print(df_impedances.head())
    Z_values = np.zeros((shape,shape), dtype=complex)
    Y_bus = np.zeros((shape,shape), dtype=complex)

    for x in range(df_impedances.shape[0]):
        from_line = df_impedances['From_line'][x]
        to_line = df_impedances['To_line'][x]

        # Adding the diagonal elements to the Y-bus
        Y_bus[from_line-1][from_line-1] += 1/(df_impedances['R'][x] + df_impedances['X'][x]*1j)
        Y_bus[from_line-1][from_line-1] += 1/2*(df_impedances['Full_line_B'][x]*1j)
        Y_bus[to_line-1][to_line-1] += 1/(df_impedances['R'][x] + df_impedances['X'][x]*1j)
        Y_bus[to_line-1][to_line-1] += 1/2*(df_impedances['Full_line_B'][x]*1j)

        # Z values for off diagonal elements
        Z_values[from_line-1][to_line-1] = df_impedances['R'][x] + df_impedances['X'][x]*1j
        Z_values[to_line-1][from_line-1] = df_impedances['R'][x] + df_impedances['X'][x]*1j
    # Adding off diagonal elements
    for i in range(shape):
        for j in range(shape):
            if(Z_values[i][j] != 0. +0.j):
                if(i != j):
                    Y_bus[i][j] = - 1/Z_values[i][j]
            else:
                if(i != j):
                    Y_bus[i][j] = Z_values[i][j]
    return Y_bus

In [527]:
Ybus = Ybus('impedances.csv', 5)
print(Ybus)

   From_line  To_line      R      X  Full_line_B
0          1        2  0.050  0.250       0.0500
1          2        3  0.025  0.125       0.1000
2          1        4  0.020  0.200       0.0333
3          2        4  0.020  0.200       0.0333
4          4        5  0.010  0.100       0.0200
[[ 1.26428027 -8.7549989j  -0.76923077 +3.84615385j
   0.         +0.j         -0.4950495  +4.95049505j
   0.         +0.j        ]
 [-0.76923077 +3.84615385j  2.80274181-16.39730659j
  -1.53846154 +7.69230769j -0.4950495  +4.95049505j
   0.         +0.j        ]
 [ 0.         +0.j         -1.53846154 +7.69230769j
   1.53846154 -7.64230769j  0.         +0.j
   0.         +0.j        ]
 [-0.4950495  +4.95049505j -0.4950495  +4.95049505j
   0.         +0.j          1.98019802-19.7586802j
  -0.99009901 +9.9009901j ]
 [ 0.         +0.j          0.         +0.j
   0.         +0.j         -0.99009901 +9.9009901j
   0.99009901 -9.8909901j ]]


In [528]:
# Litt testkode i denne blokken...

class Network:
    def __init__(self, buses):
        self.buses = buses
    
    def get_V_vec(self):
        V_vec = []
        for x in range(len(self.buses)):
            V_vec.append(self.buses[x].V)
        return V_vec

    def get_P_vec(self):
        P_vec = []
        for x in range(len(self.buses)):
            P_vec.append(self.buses[x].P)
        return P_vec

    def get_Q_vec(self):
        Q_vec = []
        for x in range(len(self.buses)):
            Q_vec.append(self.buses[x].Q)
        return Q_vec

    def get_delta_vec(self):
        delta_vec = []
        for x in range(len(self.buses)):
            delta_vec.append(self.buses[x].delta)
        return delta_vec

    def get_bus_type_vec(self):
        bus_type_vec = []
        for x in range(len(self.buses)):
            bus_type_vec.append(self.buses[x].bus_type)
        return bus_type_vec

    def get_bus_num_vec(self):
        bus_num_vec = []
        for x in range(len(self.buses)):
            bus_num_vec.append(self.buses[x].bus_num)
        return bus_num_vec
    
    def get_V_calc(self):
        V_vec = []
        for x in range(len(self.buses)):
            if(np.isnan(self.buses[x].V)):
                V_vec.append(1)
            else:
                V_vec.append(self.buses[x].V)
        return V_vec

    def get_delta_calc(self):
        delta_vec = []
        for x in range(len(self.buses)):
            if (np.isnan(self.buses[x].delta)):
                delta_vec.append(0)
            else:
                delta_vec.append(self.buses[x].delta)
        return delta_vec
    
    
    def get_PQ_vec(self):
        PQ_vec = []
        PQ_jacobian = []
        for x in range(len(self.buses)):
            if (np.isnan(self.buses[x].P) == False):
                PQ_vec.append(self.buses[x].P)
                businfo = PQ("P", self.buses[x].bus_num)
                PQ_jacobian.append(businfo)
        for x in range(len(self.buses)):
            if (np.isnan(self.buses[x].Q) == False):
                PQ_vec.append(self.buses[x].Q)
                businfo = PQ("Q", self.buses[x].bus_num)
                PQ_jacobian.append(businfo)
        return PQ_vec, PQ_jacobian

    def get_VD_jacobian(self):
        VD_jacobian = []
        VD_vec = []

        for x in range(len(self.buses)):
            if (np.isnan(self.buses[x].delta)):
                VD_vec.append(0)
                businfo = VD("D", self.buses[x].bus_num)
                VD_jacobian.append(businfo)  

        for x in range(len(self.buses)):    
            if (np.isnan(self.buses[x].V)):
                VD_vec.append(1)
                businfo = VD("V", self.buses[x].bus_num)
                VD_jacobian.append(businfo)

        return VD_vec, VD_jacobian

    

class Buses:
    def __init__(self, P, Q, V, delta, bus_num, bus_type):
        self.P = P
        self.Q = Q
        self.V = V
        self.delta = delta
        self.bus_num = bus_num 
        self.bus_type = bus_type



In [529]:
def read_buses(file):
    bus_vec = []
    df_buses = pd.read_csv(file, sep=";")
    for x in range(df_buses.shape[0]):
        V = df_buses['V'][x]
        delta = df_buses['Angle'][x]
        P = df_buses['P_gen'][x]
        Q = df_buses['Q_gen'][x]
        bus_type = df_buses['BusType'][x]
        bus_num = df_buses['BusNum'][x]
        new_bus = Buses(P, Q, V, delta, bus_num, bus_type)
        bus_vec.append(new_bus)
        
    return bus_vec

bus_vec = read_buses('Busdata.csv')

power_network = Network(bus_vec)

V_init = power_network.get_V_calc()
delta_init = power_network.get_delta_calc()
P_init = power_network.get_P_vec()
Q_init = power_network.get_Q_vec()
bus_num_init = power_network.get_bus_num_vec()

#Function to caluculate the known P values
def P_Calc(V, YBus, BusNum, delta, P):
    P_Calc = np.zeros(len(BusNum), dtype=complex)
    for i in range (len(BusNum)):
        if (np.isnan(P[i])):
            P_Calc[i] = np.nan
        else:
            for j in range (len(BusNum)):
                P_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.cos(delta[i]-delta[j])
                            +np.imag(YBus)[i][j]*cmath.sin(delta[i]-delta[j]))) 
    return P_Calc


#Function to caluculate the known Q values
def Q_Calc(V, YBus, BusNum, delta, Q):
    Q_Calc = np.zeros(len(BusNum), dtype=complex)
    for i in range (len(BusNum)):
        if (np.isnan(Q[i])):
            Q_Calc[i] = np.nan
        else:
           for j in range (len(BusNum)):
                Q_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.sin(delta[i]-delta[j])
                                         -np.imag(YBus)[i][j]*cmath.cos(delta[i]-delta[j]))) 
    return Q_Calc




P_calc = P_Calc(V_init, Ybus, bus_num_init, delta_init, P_init)
Q_calc = Q_Calc(V_init, Ybus, bus_num_init, delta_init, Q_init)

print(P_calc)
print(Q_calc)

""""
def get_PQ_calc(P_Calc, Q_Calc):
    PQ_calc = []
    for x in range(len(P_calc)):
        if (np.isnan(P_calc[x]) == False):
            PQ_calc.append(P_calc[x])
    for x in range(len(Q_calc)):
        if (np.isnan(Q_calc[x]) == False):
            PQ_calc.append(Q_calc[x])
    return PQ_calc

"""

def get_PQ_calc(P_calculated, Q_calculated):
    PQ_calc = []
    for x in range(len(P_calculated)):
        if (np.isnan(P_calculated[x]) == False):
            PQ_calc.append(P_calculated[x])
            #PQ_calc[count] = P_calculated[x]

    for x in range(len(Q_calculated)):
        if (np.isnan(Q_calculated[x]) == False):
            PQ_calc.append(Q_calculated[x])
            #PQ_calc[count] = Q_calculated[x]
    return PQ_calc

PQ_calc = get_PQ_calc(P_calc, Q_calc)  
print(PQ_calc)  
print(bus_num_init)  
print(V_init) 

[5.55111512e-17+0.j 5.55111512e-17+0.j            nan+0.j
 0.00000000e+00+0.j 0.00000000e+00+0.j]
[     nan+0.j -0.09165+0.j      nan+0.j -0.0433 +0.j -0.01   +0.j]
[(5.551115123125783e-17+0j), (5.551115123125783e-17+0j), 0j, 0j, (-0.09164999999999868+0j), (-0.043300000000002115+0j), (-0.009999999999999787+0j)]
[0, 1, 2, 3, 4]
[1.0, 1, 1.0, 1, 1]


In [530]:
"""

#Function to caluculate the known P values
def P_Calc(V, YBus, BusNum, delta, P):
    P_Calc = np.zeros(BusNum, dtype=float)
    for i in range (BusNum):
        if (np.isnan(P[i])):
            P_Calc[i] = np.nan
        else:
            for j in range (BusNum):
                P_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.cos(delta[i]-delta[j])
                            +np.imag(YBus)[i][j]*cmath.sin(delta[i]-delta[j]))) 
    return P_Calc


#Function to caluculate the known Q values
def Q_Calc(V, YBus, BusNum, delta, Q):
    Q_Calc = np.zeros(BusNum, dtype=float)
    for i in range (BusNum):
        if (np.isnan(Q[i])):
            Q_Calc[i] = np.nan
        else:
           for j in range (BusNum):
                Q_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.sin(delta[i]-delta[j])
                                         -np.imag(YBus)[i][j]*cmath.cos(delta[i]-delta[j]))) 
    return Q_Calc




P_calc = (P_Calc(V_calc, Ybus, num_buses, delta_calc, P))
Q_calc = (Q_Calc(V_calc, Ybus, num_buses, delta_calc, Q))

print(P_calc)
print(Q_calc)

"""

'\n\n#Function to caluculate the known P values\ndef P_Calc(V, YBus, BusNum, delta, P):\n    P_Calc = np.zeros(BusNum, dtype=float)\n    for i in range (BusNum):\n        if (np.isnan(P[i])):\n            P_Calc[i] = np.nan\n        else:\n            for j in range (BusNum):\n                P_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.cos(delta[i]-delta[j])\n                            +np.imag(YBus)[i][j]*cmath.sin(delta[i]-delta[j]))) \n    return P_Calc\n\n\n#Function to caluculate the known Q values\ndef Q_Calc(V, YBus, BusNum, delta, Q):\n    Q_Calc = np.zeros(BusNum, dtype=float)\n    for i in range (BusNum):\n        if (np.isnan(Q[i])):\n            Q_Calc[i] = np.nan\n        else:\n           for j in range (BusNum):\n                Q_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.sin(delta[i]-delta[j])\n                                         -np.imag(YBus)[i][j]*cmath.cos(delta[i]-delta[j]))) \n    return Q_Calc\n\n\n\n\nP_calc = (P_Calc(V_calc, Ybus, num_buses,

In [531]:
#Jacobian calculations

class PQ:
    def __init__(self, Bus_type, Bus_num):
        self.Bus_type = Bus_type
        self.Bus_num = Bus_num

class VD:
    def __init__(self, Bus_type, Bus_num):
        self.Bus_type = Bus_type
        self.Bus_num = Bus_num

""" 
def jacobian_init(P,Q, V, delta):

    len_vec = len(P) + len(Q)
    len_vec -= np.isnan(P).sum()
    len_vec -= np.isnan(Q).sum()
    print(len_vec)

    PQ_vec =  np.zeros(len_vec, dtype=float)
    PQ_jacobian = []
    VD_jacobian = []
    numP = 0
    numQ = 0
    rowP = 0
    rowQ = 0
    for x in range(len(P)):
        if (np.isnan(P[x]) == False):
            PQ_vec[numP] = P[x]
            businfo = PQ("P", rowP, numP)
            PQ_jacobian.append(businfo)
            numP += 1
        rowP += 1
    for x in range(len(Q)):
        if (np.isnan(Q[x]) == False):
            PQ_vec[numP + numQ] = Q[x]
            businfo = PQ("Q", rowQ, numQ+numP)
            PQ_jacobian.append(businfo)
            numQ += 1
        rowQ += 1

    numV = 0
    numT = 0
    rowV = 0
    rowT = 0
    for x in range(len(delta)):
        if (np.isnan(delta[x])):
            businfo = VD("T", rowT, numT)
            VD_jacobian.append(businfo)
            numT += 1
        rowT += 1    

    for x in range(len(V)):    
        if (np.isnan(V[x])):
            businfo = VD("V", rowV, numV)
            VD_jacobian.append(businfo)
            numV += 1
        rowV += 1

    return VD_jacobian, PQ_jacobian, PQ_vec
"""


PQ_vec, PQ_jacobian = power_network.get_PQ_vec()

VD_vec, VD_jacobian = power_network.get_VD_jacobian()




def make_jacobian(VD_jacobian, PQ_jacobian, PQ_vec, num_buses, V, delta):
    j = np.zeros((7,7), dtype=complex)

    #V = [1,1,1,1,1]
    #delta = [0,0,0,0,0]
    
    for x in range(len(PQ_vec)):
        for y in range(len(PQ_vec)):
            if (PQ_jacobian[x].Bus_num == VD_jacobian[y].Bus_num):
                if (PQ_jacobian[x].Bus_type == 'P' and VD_jacobian[y].Bus_type == 'D'):
                    for i in range(num_buses):
                        if (i==PQ_jacobian[x].Bus_num):
                            j[x,y] += 0
                        else:
                            j[x,y] += V[PQ_jacobian[x].Bus_num]*(-np.real(Ybus[PQ_jacobian[x].Bus_num,i])*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[i])+np.imag(Ybus[PQ_jacobian[x].Bus_num,i])*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[i]))      
                if (PQ_jacobian[x].Bus_type == 'P' and VD_jacobian[y].Bus_type == 'V'):
                    for i in range(num_buses):
                        if (i==PQ_jacobian[x].Bus_num):
                            j[x,y] += 2*V[PQ_jacobian[x].Bus_num]*np.real(Ybus[PQ_jacobian[x].Bus_num,i])
                        else:
                            j[x,y] += V[PQ_jacobian[x].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,i])*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[i])+np.imag(Ybus[PQ_jacobian[x].Bus_num,i])*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[i]))      

                if (PQ_jacobian[x].Bus_type == 'Q' and VD_jacobian[y].Bus_type == 'D'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*V[y]*(np.real(Ybus[PQ_jacobian[x].Bus_num,i]*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[i]))+np.imag(Ybus[PQ_jacobian[x].Bus_num,i]*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[i])))

                if (PQ_jacobian[x].Bus_type == 'Q' and VD_jacobian[y].Bus_type == 'V'):
                    j[x,y] += 2*V[PQ_jacobian[x].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,i])*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[i])-np.imag(Ybus[PQ_jacobian[x].Bus_num,i])*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[i]))
           
            else:
                if (PQ_jacobian[x].Bus_type == 'P' and VD_jacobian[y].Bus_type == 'D'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*V[VD_jacobian[y].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num])*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num])-np.imag(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num])*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num]))
                
                if (PQ_jacobian[x].Bus_type == 'P' and VD_jacobian[y].Bus_type == 'V'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num])*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num])+np.imag(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num])*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num]))
                
                if (PQ_jacobian[x].Bus_type == 'Q' and VD_jacobian[y].Bus_type == 'V'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*np.real(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num]*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num])-np.imag(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num]*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num])))
                
                if (PQ_jacobian[x].Bus_type == 'Q' and VD_jacobian[y].Bus_type == 'D'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*V[VD_jacobian[y].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num]*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num]))-np.imag(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num]*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num])))
    return j



j = make_jacobian(VD_jacobian, PQ_jacobian, PQ_vec, len(bus_num_init), V_calc, delta_calc)

print(j)


    

[[  8.7966489 +0.j  -3.84615385+0.j  -4.95049505+0.j   0.        +0.j
   -0.76923077+0.j  -0.4950495 +0.j   0.        +0.j]
 [ -3.84615385+0.j  16.48895659+0.j  -4.95049505+0.j   0.        +0.j
    2.80274181+0.j  -0.4950495 +0.j   0.        +0.j]
 [ -4.95049505+0.j  -4.95049505+0.j  19.8019802 +0.j  -9.9009901 +0.j
   -0.4950495 +0.j   1.98019802+0.j  -0.99009901+0.j]
 [  0.        +0.j   0.        +0.j  -9.9009901 +0.j   9.9009901 +0.j
    0.        +0.j  -0.99009901+0.j   0.99009901+0.j]
 [ -0.76923077+0.j   0.        +0.j  -0.4950495 +0.j   0.        +0.j
    0.        +0.j  -4.95049505+0.j   0.        +0.j]
 [ -0.4950495 +0.j  -0.4950495 +0.j  -0.99009901+0.j  -0.99009901+0.j
   -4.95049505+0.j -19.8019802 +0.j  -9.9009901 +0.j]
 [  0.        +0.j   0.        +0.j  -0.99009901+0.j   0.99009901+0.j
    0.        +0.j  -9.9009901 +0.j  19.7819802 +0.j]]


In [532]:
# Inverse of jacobian
j_inv = np.linalg.inv(j)
print(j_inv)

[[ 0.28031036+0.j  0.12657994+0.j  0.20304869+0.j  0.2034404 +0.j
  -0.03914098+0.j  0.00780293+0.j  0.00388581+0.j]
 [ 0.08973909+0.j  0.1125314 +0.j  0.09911024+0.j  0.10111099+0.j
  -0.19992009+0.j  0.03985498+0.j  0.0198475 +0.j]
 [ 0.20146015+0.j  0.12716129+0.j  0.2656131 +0.j  0.26632232+0.j
  -0.05046334+0.j  0.01412774+0.j  0.00703551+0.j]
 [ 0.19826227+0.j  0.12553556+0.j  0.26269584+0.j  0.3649065 +0.j
  -0.06004523+0.j  0.01399595+0.j  0.00188935+0.j]
 [ 0.20130453+0.j  0.08734302+0.j  0.15444881+0.j  0.14444507+0.j
   0.99960044+0.j -0.19927492-0.j -0.0992375 -0.j]
 [-0.06370193-0.j -0.0323847 -0.j -0.05811195-0.j -0.05824374-0.j
  -0.19087176-0.j -0.00262523-0.j -0.00130735-0.j]
 [-0.03172311-0.j -0.01612735-0.j -0.02893934-0.j -0.03408551-0.j
  -0.09505279-0.j -0.00130735-0.j  0.05015429+0.j]]


In [533]:

def delta_VD(PQ_vec, PQ_calc, j_inv):
    delta_PQ = np.array(PQ_vec) - np.array(PQ_calc)
    delta_VD = np.matmul(j_inv,delta_PQ)
    return delta_VD

delta_vd = delta_VD(PQ_vec, PQ_calc, j_inv)

print(delta_vd)


print(VD_vec)



[-0.01377018+0.j -0.06013383+0.j -0.16180928+0.j -0.20754941+0.j
 -0.15433086+0.j  0.0604073 +0.j  0.01280859+0.j]
[0, 0, 0, 0, 1, 1, 1]


In [534]:

def updateVD(VD_vec, delta_vd):
    VD_vec_updated = VD_vec.copy()
    VD_vec_updated = np.array(VD_vec) + np.array(delta_vd)
    return VD_vec_updated

VD_vec_current = updateVD(VD_vec, delta_vd)

print(VD_vec_current)



[-0.01377018+0.j -0.06013383+0.j -0.16180928+0.j -0.20754941+0.j
  0.84566914+0.j  1.0604073 +0.j  1.01280859+0.j]


In [535]:
def updateVD_vec(VD_vec_current,delta_current,V_current):
    delta_updated = delta_current.copy()
    delta_updated = np.array(delta_updated, complex)
    V_updated = V_current.copy()
    V_updated = np.array(V_updated, complex)
    c = 0
    for x in range(len(delta_updated)):
        if (np.isnan(delta_updated[x])):
            delta_updated[x] = VD_vec_current[c]
            c += 1
    for x in range(len(V_updated)):
        if (np.isnan(V_updated[x])):
            V_updated[x] = VD_vec_current[c]
            c += 1
    return delta_updated, V_updated

delta_updated, V_updated = updateVD_vec(VD_vec_current,delta,V)
print(V)
print(delta_updated)
print(V_updated)

def updatePQ_vec(PQ_vec, V_current, delta_current, Ybus, bus_num_init, P_init, Q_init):
    PQ_vec_updated = PQ_vec.copy() #Hvorfor gjør vi dette?
    P_calc_updated = P_Calc(V_current, Ybus, bus_num_init, delta_current, P_init)
    Q_calc_updated = Q_Calc(V_current, Ybus, bus_num_init, delta_current, Q_init)
    print(P_calc_updated)
    print(Q_calc_updated)
    PQ_vec_updated = get_PQ_calc(P_calc_updated, Q_calc_updated) 
    return PQ_vec_updated

PQ_vec_updated = updatePQ_vec(PQ_vec, V_updated, delta_updated, Ybus, bus_num_init, P_init, Q_init)

print(PQ_vec_updated)
  

[ 1. nan  1. nan nan]
[-0.01377018+0.j -0.06013383+0.j  0.        +0.j -0.16180928+0.j
 -0.20754941+0.j]
[1.        +0.j 0.84566914+0.j 1.        +0.j 1.0604073 +0.j
 1.01280859+0.j]
[ 1.02030136+0.j -0.47683259+0.j         nan+0.j -0.53512941+0.j
 -0.53282977+0.j]
[        nan+0.j -2.36899995+0.j         nan+0.j  2.0607769 +0.j
 -0.42782581+0.j]
[(1.020301362085072+0j), (-0.4768325897867542+0j), (-0.535129407233201+0j), (-0.532829767641416+0j), (-2.3689999548776823+0j), (2.0607768994680953+0j), (-0.4278258069326881+0j)]


In [536]:
#Function to caluculate updated P values
def P_Updated(V, YBus, BusNum, delta):
    P_Updated = np.zeros(len(BusNum), dtype=complex)
    for i in range (len(BusNum)):
        for j in range (len(BusNum)):
            P_Updated[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.cos(delta[i]-delta[j])
                        +np.imag(YBus)[i][j]*cmath.sin(delta[i]-delta[j]))) 
    return P_Updated

#Function to caluculate updated Q values
def Q_Updated(V, YBus, BusNum, delta):
    Q_updated = np.zeros(len(BusNum), dtype=complex)
    for i in range (len(BusNum)):
        for j in range (len(BusNum)):
            Q_updated[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.sin(delta[i]-delta[j])
                         -np.imag(YBus)[i][j]*cmath.cos(delta[i]-delta[j]))) 
    return Q_updated
Q_updated=Q_Updated(V_updated, Ybus, bus_num_init, delta_updated)
P_updated=P_Updated(V_updated,Ybus, bus_num_init, delta_updated)
print(P_updated)
print(Q_updated)


#Check for violations
def Q_max_violation(Q_Updated, Q_max):
    Q_Updated = [0.75, -2.37, 1.07, 2.06, -0.43]
    for i in range (len(Q_max)):
        if Q_max[i] == '':
            continue
        if Q_max[i] < Q_Updated[i]:
            print('Q_max is violated for bus ', bus_num_init[i+1], 'and needs to be type switched.')
            bus_type[i] = 2
            Q_Updated[i] = Q_max[i]
            V[i] = np.nan
            print(V)
            print(bus_type)  
        else:
            print('Bus', bus_num_init[i+1], 'is within its boundaries.')
            continue
    return Q_Updated

print(Q_max_violation(Q_Updated(V_updated, Ybus, bus_num_init, delta_updated), Q_max))



[ 1.02030136+0.j -0.47683259+0.j  0.63072739+0.j -0.53512941+0.j
 -0.53282977+0.j]
[ 0.20621815+0.j -2.36899995+0.j  1.0707297 +0.j  2.0607769 +0.j
 -0.42782581+0.j]
Q_max is violated for bus  1 and needs to be type switched.
[nan nan  1. nan nan]
[2 2 0 2 2]
[0.5, -2.37, 1.07, 2.06, -0.43]


In [537]:

j_new = make_jacobian(VD_jacobian, PQ_jacobian, PQ_vec_updated, len(bus_num_init), V_updated, delta_updated)

print(j_new)


[[ 8.84703920e+00+0.j -3.27922784e+00+0.j -5.26955291e+00+0.j
   0.00000000e+00+0.j -5.90146296e-01+0.j  2.40558133e-01+0.j
   0.00000000e+00+0.j]
 [-3.21892898e+00+0.j  1.38414823e+01+0.j -4.46150760e+00+0.j
   0.00000000e+00+0.j  2.25864344e+00+0.j  8.44330457e-03+0.j
   0.00000000e+00+0.j]
 [-5.11469253e+00+0.j -4.37138798e+00+0.j  2.08199468e+01+0.j
  -1.06710601e+01+0.j -1.05507326e+00+0.j  1.28229729e+00+0.j
  -5.68748182e-01+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j -1.05738180e+01+0.j
   9.97146847e+00+0.j  0.00000000e+00+0.j -1.46024529e+00+0.j
   5.45316284e-01+0.j]
 [-4.99068513e-01+0.j  0.00000000e+00+0.j -8.92242899e-01+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j -4.20735277e+00+0.j
   0.00000000e+00+0.j]
 [ 2.55089601e-01+0.j  8.95334181e-03+0.j -5.68748182e-01+0.j
  -1.54845477e+00+0.j -5.16914683e+00+0.j -2.10722148e+01+0.j
  -1.05361074e+01+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j -5.76033046e-01+0.j
   1.06335607e+00+0.j  0.00000000e+00+0.j -9.97146847e+0

In [538]:
# Inverse of jacobian
j_new_inv = np.linalg.inv(j_new)
print(j_new_inv)

[[ 0.34620066+0.j  0.17075274+0.j  0.28091787+0.j  0.298435  +0.j
   0.14171362+0.j -0.02225288+0.j -0.01185048+0.j]
 [ 0.10052298+0.j  0.13003807+0.j  0.1076297 +0.j  0.11751522+0.j
  -0.14784069+0.j  0.02337507+0.j  0.01214919+0.j]
 [ 0.2768129 +0.j  0.17646861+0.j  0.35106022+0.j  0.37311626+0.j
   0.15769157+0.j -0.02615029+0.j -0.01394154+0.j]
 [ 0.28202286+0.j  0.18049844+0.j  0.35991059+0.j  0.4831026 +0.j
   0.13302708+0.j -0.02679072+0.j -0.01702067+0.j]
 [ 0.42452627+0.j  0.23798547+0.j  0.4346272 +0.j  0.44260383+0.j
   1.42053123+0.j -0.22664704-0.j -0.11889695-0.j]
 [-0.09976872-0.j -0.05767764-0.j -0.10777044-0.j -0.11452566-0.j
  -0.28793022-0.j  0.00818522+0.j  0.00436223+0.j]
 [-0.05666372-0.j -0.03321194-0.j -0.06264526-0.j -0.07191145-0.j
  -0.14582751-0.j  0.00474378+0.j  0.05258533+0.j]]


In [539]:
#printing nicely
from pandas import *

# printing Ybus
df = DataFrame(Ybus)
df.index = np.arange(1, len(df)+1)
df.columns = np.arange(1, len(df)+1)
print('Ybus: \n', df, '\n')

#printing Jacobian
df1 = DataFrame(np.real(j))
df1.index = np.arange(1, len(df1)+1)
df1.columns = np.arange(1, len(df1)+1)
print('Jacobian: \n', df1, '\n')

def buses(V_updated, delta_updated, P_updated, Q_updated):
    table = []
    for i in range (len(bus_num_init)):
        table.append([bus_num_init[i]+1, bus_type[i], np.real(V_updated[i]), np.real(delta_updated[i]), np.real(P_updated[i]), np.real(Q_updated[i])])
    return table

print(buses(V_updated, delta_updated, P_updated, Q_updated))




Ybus: 
                     1                    2                   3  \
1  1.264280-8.754999j  -0.769231+3.846154j  0.000000+0.000000j   
2 -0.769231+3.846154j  2.802742-16.397307j -1.538462+7.692308j   
3  0.000000+0.000000j  -1.538462+7.692308j  1.538462-7.642308j   
4 -0.495050+4.950495j  -0.495050+4.950495j  0.000000+0.000000j   
5  0.000000+0.000000j   0.000000+0.000000j  0.000000+0.000000j   

                     4                   5  
1  -0.495050+4.950495j  0.000000+0.000000j  
2  -0.495050+4.950495j  0.000000+0.000000j  
3   0.000000+0.000000j  0.000000+0.000000j  
4  1.980198-19.758680j -0.990099+9.900990j  
5  -0.990099+9.900990j  0.990099-9.890990j   

Jacobian: 
           1          2          3         4         5          6          7
1  8.796649  -3.846154  -4.950495  0.000000 -0.769231  -0.495050   0.000000
2 -3.846154  16.488957  -4.950495  0.000000  2.802742  -0.495050   0.000000
3 -4.950495  -4.950495  19.801980 -9.900990 -0.495050   1.980198  -0.990099
4  0.00